# Nano (cryptocurrency) for dummies

This tutorial is intended to complement the [official Nano documentation](https://docs.nano.org/integration-guides/the-basics/). It implements almost all important functions needed to create valid blocks. The code was written in a simply and intuitive manner so that it's easy to follow.

By reading this tutorial you will understand how a seed is created and how private/public key pairs are derived from it. These in turn are used to send and receive funds. You will understand how to build Nano blocks and what needs to be done for the network to consider them valid. Finally, once funds have been sent and received, we will import the seed to a popular wallet which will help us reinforce the fact that the seed is the key to any funds. 

Go to the beginning of the tutorial **[here](#tutorial-start)**

In [ ]:
import re
import binascii
import secrets
import base64
import time
import random
import json
import pprint
from hashlib import blake2b

try:
    from typing import TypedDict
except ImportError:
    from mypy_extensions import TypedDict

import nanolib as nl
import requests
import ed25519_blake2b

In [ ]:
SEED_SIZE_BYTES = 32
KEYS_SIZE_BYTES = 32
CHECKSUM_DIGEST_SIZE_BYTES = 5
PRIVATE_KEY_INDEX_SIZE_BYTES = 4
SIGNATURE_DIGEST_SIZE_BYTES = 32
WORK_SIZE_BYTES = 8
BALANCE_SIZE_BYTES = 16
NANO_ADDRESS_REGEX = '^(nano|xrb)_[13]{1}[13456789abcdefghijkmnopqrstuwxyz]{59}$'

MAX_NONCE_SIZE = 2 ** 64 - 1
MINIMUM_NETWORK_DIFFICULTY = "fffffff800000000"
MINIMUM_RECEIVE_NETWORK_DIFFICULTY = "fffffe0000000000"

DEFAULT_NANO_NODE = 'https://mynano.ninja/api/node'

The block specification can be found [here](https://docs.nano.org/integration-guides/the-basics/#blocks-specifications).

In [ ]:
class Block(TypedDict):
    type: str
    account: str
    previous: str
    representative: str
    balance: str
    link: str
    signature: str
    work: str

In [ ]:
def generate_seed() -> str:
    '''
    Creates a new seed from bits produced by the system's strong random numbers generator.
    Returns it in base16 encoding (using hexadecimal characters).
    
    More info: https://docs.nano.org/integration-guides/the-basics/#seed
    '''
    seed = secrets.token_bytes(SEED_SIZE_BYTES)
    return base64.b16encode(seed).decode().upper()

In [ ]:
def generate_public_private_key_pairs(seed: str, index: int = 0) -> (str, str):
    '''
    Using a base16 encoded seed and 32-bit number, we create a private key and its
    corresponding public key. Both are returned in base16 encoding.
    The formula for the private key is: private_key = Blake2b(seed || seed)
    
    More info: https://docs.nano.org/integration-guides/the-basics/#account-private-key
    '''
    if index > 2 ** 32 - 1:
        raise Exception('Index must be representable with 32 bits. Max: {}'.format(2 ** 32  - 1))
    # Regular Python ints are arbitrarily long, we want a 32-bits int, so we truncate. 
    index_in_bytes = int(index).to_bytes(8, byteorder='big')[:-PRIVATE_KEY_INDEX_SIZE_BYTES]
    hasher = blake2b(digest_size=KEYS_SIZE_BYTES)
    hasher.update(binascii.unhexlify(seed))
    hasher.update(index_in_bytes)
    private_key = hasher.digest()
    signing_key = ed25519_blake2b.SigningKey(private_key)
    verifying_key = signing_key.get_verifying_key().to_bytes()
    return binascii.hexlify(private_key).upper().decode(), binascii.hexlify(verifying_key).upper().decode()

In [ ]:
def derive_address_from_public_key(public_key: str) -> str:
    '''
    Receives a base16 encoded public key and returns a corresponding Nano address
    of the form: nano_16odwi933gpzmkgdcy9tt5zef5ka3jcfubc97fwypsokg7sji4mb9n6qtbme
    
    More info: https://docs.nano.org/integration-guides/the-basics/#account-public-address
    '''
    key = binascii.unhexlify(public_key)
    base64_key = nl.bytes_to_nbase32(key)
    checksum = bytearray(blake2b(key, digest_size=CHECKSUM_DIGEST_SIZE_BYTES).digest())
    checksum.reverse()
    base64_checksum = nl.bytes_to_nbase32(bytes(checksum))
    return 'nano_{}{}'.format(base64_key, base64_checksum)

def derive_public_key_from_address(address: str) -> str:
    '''
    Receives a Nano address of the form: nano_16odwi933gpzmkgdcy9tt5zef5ka3jcfubc97fwypsokg7sji4mb9n6qtbme
    and returns the base16 encoded public key corresponding to it.
    
    More info: https://docs.nano.org/integration-guides/the-basics/#account-public-address
    '''
    if not re.match(NANO_ADDRESS_REGEX, address):
        raise Exception('Invalid Nano address: {}'.format(address))
    address_part = address[5:-8]
    checksum = address[-8:]
    public_key = nl.nbase32_to_bytes(address_part)
    checksum_bits = bytearray(blake2b(public_key, digest_size=5).digest())
    checksum_bits.reverse()
    computed_checksum = nl.bytes_to_nbase32(bytes(checksum_bits))
    if checksum != computed_checksum:
        raise Exception("Invalid Nano address. Could not validate checksum.")
    return binascii.hexlify(public_key).decode().upper()

In [ ]:
# This was formerly used to include the type of block in the signature.
# However, since the migration to Universal Blocks, the type of all blocks is 'state',
# thus this remains for backwards compatibility, but is no longer useful.
STATE_BLOCK_HEADER_BYTES = binascii.unhexlify("0000000000000000000000000000000000000000000000000000000000000006")

def process_link_data(block: Block) -> str:
    '''
    Each block has a link attribute which contains different data depending
    on the purpose that the block is serving:
        - SEND: contains the address of the recipient.
        - RECEIVE: contains the block hash of the matching SEND block.
        - CHANGE: contains a string with 0s.
        
    More info: https://docs.nano.org/integration-guides/the-basics/#block-format
    '''
    if re.match(NANO_ADDRESS_REGEX, block['link']):
        # it's a send block
        return derive_public_key_from_address(block['link'])
    elif re.match('^[a-fA-F0-9]+$', block['link']):
        # it's a receive block
        return block['link']
    elif len(set(block['link'])) == 1 and '0' in block['link']:
        # it's a change block
        return block['link']
    else:
        raise Exception('Block type cannot be determined from link attribute: {}'.format(block['link']))

def create_block_hash(block: Block) -> str:
    '''
    Given a Block, generates its hash by serializing its fields, concatenating them,
    and computing the blake2b hash of the result. Returns the hash in byte string.
    
    More info: https://docs.nano.org/integration-guides/the-basics/#self-signed-blocks
    '''
    public_key = derive_public_key_from_address(block['account'])
    representative_key = derive_public_key_from_address(block['representative'])
    
    account = binascii.unhexlify(public_key)
    previous = binascii.unhexlify(block['previous'])
    representative = binascii.unhexlify(representative_key)
    balance = int(block['balance']).to_bytes(BALANCE_SIZE_BYTES, byteorder='big')
    link = binascii.unhexlify(process_link_data(block))
    
    hasher = blake2b(digest_size=SIGNATURE_DIGEST_SIZE_BYTES)
    hasher.update(b''.join([
        STATE_BLOCK_HEADER_BYTES, account, previous, representative, balance, link
    ]))
    return hasher.digest()

def create_block_signature(block: Block, private_key: str) -> str:
    '''
    Receives a block and a base16 encoded private key, computes the hash of the block,
    and then signs it using the private key. Returns the base16 encoded signature.
    
    More info: https://docs.nano.org/integration-guides/the-basics/#self-signed-blocks
    '''
    block_hash_bits = create_block_hash(block)
    signing_key = ed25519_blake2b.SigningKey(binascii.unhexlify(private_key))
    signature_bits = signing_key.sign(block_hash_bits)
    return binascii.hexlify(signature_bits).decode().upper()

def validate_block_signature(block: Block, signature: str, address: str) -> bool:
    '''
    Validates whether the signature for the block was created with a private key
    matching the base16 encoded public key provided. Returns true or false depending
    on whether the signature is valid or not. 
    
    More info: https://docs.nano.org/integration-guides/the-basics/#self-signed-blocks
    '''
    public_key = derive_public_key_from_address(address)
    verifying_key = ed25519_blake2b.VerifyingKey(public_key, encoding='hex')
    block_hash = create_block_hash(block)
    signature_bits = binascii.unhexlify(signature)
    try:
        verifying_key.verify(signature_bits, block_hash)
        print('Signature is valid.')
        return True
    except ed25519_blake2b.BadSignatureError as e:
        print('Signature is invalid: {}'.format(str(e)))
        return False

In [ ]:
def get_block_hash(block: Block) -> str:
    if len(set(block['previous'])) == 1 and '0' in block['previous']:
        public_key = derive_public_key_from_address(block['account'])
        return public_key
    else:
        return block['previous']

#
#  WARNING: This function is extremely inefficient. It will take several minutes to compute the PoW at default difficulty.
#
def compute_work(block: Block, difficulty: str = MINIMUM_NETWORK_DIFFICULTY, timeout_seconds: int = 720) -> str:
    '''
    Given a block and a network difficulty, this function calculates the nonce necessary to
    submit the block to the network. It receives as a parameter a timeout. The function will
    continue looking for a nonce until it is found, or the timeout is exceeded. In the latter case,
    None is returned.
    
    The nonce returned satisfies the equation:
    1. For the first block in an account: 
        Blake2b(nonce || public_key) >= network difficulty
        
    2. For all other blocks: 
        Blake2b(nonce || previous_block) >= network_difficulty
    
    More info: https://docs.nano.org/integration-guides/work-generation/#work-calculation-details
    '''
    block_hash = binascii.unhexlify(get_block_hash(block))
    nonce = None
    start_time = time.process_time()
    while time.process_time() - start_time < timeout_seconds:
        candidate = bytearray(random.randint(0, MAX_NONCE_SIZE).to_bytes(8, byteorder='big'))
        candidate.reverse()
        work = blake2b(b''.join([candidate, block_hash]), digest_size=WORK_SIZE_BYTES).digest()
        work = bytearray(work)
        work.reverse()
        computed_work = binascii.hexlify(work).decode()
        if computed_work >= difficulty:
            candidate.reverse()
            nonce = binascii.hexlify(candidate)
            break
    if nonce:
        end_time = time.process_time()
        print('Took {}s seconds to compute work.'.format(end_time - start_time))
        return nonce.decode()
    else:
        print('Work computation timed out. No work found for the selected difficulty.')

def validate_work(block: Block, nonce: str, difficulty: str = MINIMUM_NETWORK_DIFFICULTY) -> True:
    '''
    Receives a block and a nonce, validates whether the nonce satisfies the PoW equation at the provided
    level of network difficulty.
    
    More info: https://docs.nano.org/integration-guides/work-generation/#work-calculation-details
    '''
    if not nonce:
        print('Invalid nonce provided.')
        return False
    block_hash = binascii.unhexlify(get_block_hash(block))
    bin_nonce = bytearray(binascii.unhexlify(nonce))
    bin_nonce.reverse()
    computed_hash = bytearray(blake2b(b''.join([bin_nonce, block_hash]), digest_size=WORK_SIZE_BYTES).digest())
    computed_hash.reverse()
    work = binascii.hexlify(computed_hash).decode()
    if work >= difficulty:
        print('Nonce {} is valid. Hash computed is {} which is larger than the difficulty {}.'.format(nonce, work, difficulty))
        return True
    else:
        print('Nonce {} is not valid. Computed work is {} which is smaller than difficulty {}.'.format(nonce, work, difficulty))
        return False

In [ ]:

def process_block(block: Block,
                  node_hostname: str,
                  subtype: str,
                  difficulty: str = MINIMUM_NETWORK_DIFFICULTY) -> any:
    if not validate_work(block, block['work']):
        raise Exception('Work used in block is invalid.')
    if not validate_block_signature(block, block['signature'], block['account']):
        raise Exception('Signature used in block is invalid.')
    data = {'subtype': subtype, 'action': 'process', 'block': block, 'json_block': True}
    print('Sending data to node:')
    pprint.PrettyPrinter(indent=2).pprint(data)
    r = requests.post(DEFAULT_NANO_NODE, json=data)
    return r.json()

<a id='tutorial-start'></a>


# Tutorial Start

### 1. Deriving a Nano address
Nano addresses are derived using a private key which is generated through a random seed. The seed is what gives you access to your account and proves your ownership of it to the network. Whoever has access to a seed, has control over the accounts based on that seed.


Execute the cell below and use the resulting Nano address to have a faucet or wallet send funds to it.

One possible faucet would be: https://www.freenanofaucet.com/

In [ ]:
index = 0
seed = generate_seed()
private_key, public_key = generate_public_private_key_pairs(seed, index)
public_address = derive_address_from_public_key(public_key)
print('Nano address: {}'.format(public_address))

### 2. Validate that funds were sent to your address

Using a Nano block explorer (i.a. https://nanocrawler.cc/), do a lookup of the address you used to find all SEND blocks pending for confirmation. You should see one block, which is the one that was generated by the faucet. In order to pocket these Nanos, you need to create a corresponding RECEIVE block, so that the network acknowledges your reception of the funds. We will do that in the following cells.

We will need some information from the block explorer:
1. The **balance**, that is, the amount Nano sent. This is a very big number (at most 16 bytes long, that is 128 bits), the unit used for all balances is the raw, the smallest unit representable by the network. One Nano is equivalent to 10 ** 30 raw. 
2. The **block hash**, the unique identifier that represents this block in the network.

### 3. Block creation

There are three types of blocks SEND, RECEIVE and CHANGE. Every account in Nano is created from a RECEIVE block, which is the one we'll create in order to pocket the money from the faucet. The very first block for this account.

You can find information on what each of the block fields represent in these links:
- https://docs.nano.org/integration-guides/the-basics/#block-format
- https://docs.nano.org/integration-guides/key-management/#creating-transactions

In [ ]:
# Example: '123000000000000000000000000'
balance = '< YOUR BALANCE GOES HERE >'  

# Example: '785A65D2125806A9AFF251D60F39F1C80FD39FD1F125FACD7E6293FD33359B55'
previous = '< THE PREVIOUS BLOCK HASH GOES HERE >'

block = {
    'type': 'state',
    'account': public_address,
    'previous': '0' * 64,
    'representative': 'nano_1wenanoqm7xbypou7x3nue1isaeddamjdnc3z99tekjbfezdbq8fmb659o7t',
    'balance': balance,
    'link': previous,
}

We compute the signature of the block by obtaining the Blake2b hash of the concatenation of its fields, and then signing it using the private key that matches the Nano address used in the faucet.

In [ ]:
signature = create_block_signature(block, private_key)
validate_block_signature(block, signature, public_address)
block['signature'] = signature

**WARNING:** The *compute_work* function is extremely inefficient. It will take several minutes to compute the PoW at default difficulty.

In order to prevent spam in the network, we must calculate a small PoW for the network to accept our valid block. Highly optimized setups can
compute PoW is seconds, however, our example will take several minutes. Please have patience. 

In [ ]:
nonce = compute_work(block, timeout_seconds=1000)
validate_work(block, nonce)
block['work'] = nonce

### 4. Ask a node to process our block

Once our block has been created, properly signed and its PoW computed, all we need to do is send it to a node so that it gets propagated throughout the network. We can send this request as many times as we want, since the operation will only execute once.

In [ ]:
response = process_block(block, DEFAULT_NANO_NODE, 'open')
if 'hash' in response:
    print('Transaction was accepted. Its block hash is: {}'.format(response['hash']))
else:
    print('Transaction was not accepted. Response is: {}'.format(response))

### 5. Validate that the block was processed by the network

Using a Nano block explorer (i.a. https://nanocrawler.cc), check that the block hash obtained above was registered by the network. Once the RECEIVE block is processed, you'll have pocketed the funds and they'll be ready to be spent.

### 6. Sending Nanos
In the following cells we repeat the steps taken above, but this time to submit a SEND block.

In [ ]:
# Example: FF474514E2B7DED97A665C21210C273EFB6A6D4ED4A30EAAE761837D3FED25F3
previous = '< PREVIOUS BLOCK HASH HERE >'

recipient_nano_address = '< RECIPIENT HERE >'

send_block = {
    'type': 'state',
    'account': public_address,
    'previous': previous,
    'representative': 'nano_1wenanoqm7xbypou7x3nue1isaeddamjdnc3z99tekjbfezdbq8fmb659o7t',
    'balance': '1',
    'link': recipient_nano_address
}

In [ ]:
send_block['signature'] = create_block_signature(send_block, private_key)

**WARNING:** The *compute_work* function is extremely inefficient. It will take several minutes to compute the PoW at default difficulty.

In [ ]:
send_block['work'] = compute_work(send_block, timeout_seconds=10000)

In [ ]:
process_block(send_block, DEFAULT_NANO_NODE, 'send')

### 7. Use a Nano wallet to continue managing your account

Import the seed you created into a wallet (i.a https://nault.cc) to continue operating your account. The wallet will load the operations performed by it using only the seed. This shows that in Nano, as in any cryptocurrency worth its salt, your funds are tied to this number. 
For this reason, **it's extremely important to keep it secret and safe**.

In [ ]:
print('Use this seed to import your account into a wallet: {}'.format(seed))